## Tensorflow inference to generate heatmap of crowd

Handling Imports

In [8]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
import os
import random
import math
import sys
from pyheatmap.heatmap import HeatMap

In [13]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = keras.models.model_from_json(loaded_model_json)

model.load_weights("weights.h5")

print("Loaded model from disk successfully")

Loaded model from disk successfully


In [17]:
dataset = "B"
img_path = './data/original/shanghaitech/part_' + dataset + '_final/test_data/images/'
den_path = './data/original/shanghaitech/part_' + dataset + '_final/test_data/ground_truth_csv/'
print('loading test data from dataset', dataset, '...')
img_names = os.listdir(img_path)
img_num = len(img_names)

data = []
for i in range(1, img_num + 1):
    if i % 50 == 0:
        print(i, '/', img_num)
    name = 'IMG_' + str(i) + '.jpg'
    img = cv2.imread(img_path + name, 0)
    img = np.array(img)
    img = (img - 127.5) / 128
    den = np.loadtxt(open(den_path + name[:-4] + '.csv'), delimiter = ",")
    den_sum = np.sum(den)
    data.append([img, den_sum])

    # if i <= 2:
        # heatmap(den, i, dataset, 'act')

loading test data from dataset B ...
50 / 316
100 / 316
150 / 316
200 / 316
250 / 316
300 / 316


In [18]:
mae = 0
mse = 0

In [23]:
d = data[0]
inputs = np.reshape(d[0], [1, 768, 1024, 1])
outputs = model.predict(inputs)
den = d[1]
c_actual = np.sum(den)
c_predicted = np.sum(outputs)

1/1 [==============================] - 0s 173ms/step


In [24]:
print(c_actual)
print(c_predicted)

22.99991944
35.028015


In [26]:
y_p_den = np.reshape(outputs, (outputs.shape[1], outputs.shape[2]))

In [27]:

def heatmap(den, img_num, dataset, info):
    print('generating heat map for img', img_num)
    print('shape:', den.shape)
    
    if info == 'pre':
        den_resized = np.zeros((den.shape[0] * 4, den.shape[1] * 4))
        for i in range(den_resized.shape[0]):
            for j in range(den_resized.shape[1]):
                den_resized[i][j] = den[int(i / 4)][int(j / 4)] / 16
        den = den_resized
    
    
    img_path = './data/original/shanghaitech/part_' + dataset + '_final/test_data/images/'
    count = np.sum(den)
    den = den * 10 / np.max(den)
    img = cv2.imread(img_path + 'IMG_' + str(img_num) + '.jpg', 1)
    
    w = img.shape[1]
    h = img.shape[0]
     
    data = []
    for j in range(len(den)):
        for i in range(len(den[0])):
            for k in range(int(den[j][i])):
                data.append([i + 1, j + 1])
    hm = HeatMap(data, base = img_path + 'IMG_' + str(img_num) + '.jpg')
    hm.heatmap(save_as = 'heat_' + dataset + '_' + str(img_num) + '_' + info + '_' + str(int(count)) + '.png')

In [28]:
heatmap(y_p_den, 1, dataset, 'pre')

generating heat map for img 1
shape: (192, 256)
